In [ ]:
import requests
from xml.etree import ElementTree
from collections import defaultdict

In [ ]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
def createTree(link):
    response = requests.get(link)
    tree = ElementTree.fromstring(response.content)
    return tree

In [ ]:
def getReactionID(tree):
    return tree["ptools-xml"]["Reaction"]["@frameid"]

In [ ]:
def getProteinComplex(tree):
    return tree["ptools-xml"]["Protein"]["@frameid"]

In [ ]:
def isReversible(tree):
    return "REVERSIBLE" == tree["ptools-xml"]["Reaction"]["reaction-direction"]

In [ ]:
def getStoichiometrySubstrate(tree, geneReactions, stoichDict):
    coefficients = []
    substrates = tree["ptools-xml"]["Reaction"]["left"]
    
    if type(substrates) is dict:
        name = substrates["Compound"]["@frameid"]
        if "coefficient" in substrates:
            coef = substrates["coefficient"]["#text"]
            stoichDict[name] = int(coef) * -1
        else:
            stoichDict[name] = -1
    else:
        for compound in substrates:
            name = compound["Compound"]["@frameid"]
            if "coefficient" in compound:
                coef = compound["coefficient"]["#text"]
                stoichDict[name] = int(coef) * -1
            else:
                stoichDict[name] = -1

In [ ]:
def getStoichiometryProduct(tree, geneReactions, stoichDict):
    coefficients = []
    products = tree["ptools-xml"]["Reaction"]["right"]
    
    if type(products) is dict:
        name = products["Compound"]["@frameid"]
        if name not in stoichDict:
            if "coefficient" in products:
                coef = products["coefficient"]["#text"]
                stoichDict[name] = int(coef)
            else:
                stoichDict[name] = 1
        else:
            if "coefficient" in products:
                coef = products["coefficient"]["#text"]
                stoichDict[name] = [stoichDict[name], int(coef)]
            else:
                stoichDict[name] = [stoichDict[name], 1]
    else:
        for compound in products:
            name = compound["Compound"]["@frameid"]
            if name not in stoichDict:
                if "coefficient" in products:
                    coef = products["coefficient"]["#text"]
                    stoichDict[name] = int(coef)
                else:
                    stoichDict[name] = 1
            else:
                if "coefficient" in products:
                    coef = products["coefficient"]["#text"]
                    stoichDict[name] = [stoichDict[name], int(coef)]
                else:
                    stoichDict[name] = [stoichDict[name], 1]
                
    geneReactions["stoichiometry"] = stoichDict

In [ ]:
def createDictFromName(complexList):
    result = []
    
    for name in complexList:
        linkComplex = "https://websvc.biocyc.org/getxml?id=ECOLI:" + name + "&detail=low"
        treeComplex = createTree(linkComplex)
        treeDictComplex = etree_to_dict(treeComplex)
        result.append(treeDictComplex)
        
    return result

In [ ]:
def allMonomers(currentDicts):
    for dictionary in currentDicts:
        if "gene" not in dictionary["ptools-xml"]["Protein"]:
            return False
        
    return True

In [ ]:
def getMonomersHelper(dictionary):
    result = []
    components = dictionary["ptools-xml"]["Protein"]["component"]
    
    if type(components) is dict:
        result.append(components["Protein"]["@frameid"])         
    else:
        for protein in components:
            result.append(protein["Protein"]["@frameid"])
            
    return result
             

In [ ]:
def getMonomers(currentList, monomerDicts):
    currentDicts = createDictFromName(currentList)
    
    for dictionary in currentDicts:
        if "gene" in dictionary["ptools-xml"]["Protein"]:
            monomerDicts.append(dictionary)
        else:
            monomers = getMonomersHelper(dictionary)
            getMonomers(monomers, monomerDicts)
            
    return

In [ ]:
def getGenes(currentList):
    result = []
    monomerDicts = []
    getMonomers(currentList, monomerDicts)
    
    for dictionary in monomerDicts:
        genes = dictionary["ptools-xml"]["Protein"]["gene"]
        if type(genes) is dict:
            result.append(genes["Gene"]["@frameid"])         
        else:
            for gene in genes:
                result.append(protein["@frameid"])
                
    return result

In [ ]:
def getMonomersCountsGenes(easy):
    
    complexDicts = createDictFromName(easy)

    monomers = []
    inAddition = []
    count = []
    genes = []
    

    for dictionary in complexDicts:

        components = dictionary["ptools-xml"]["Protein"]

        if "component" in components:
            components = components["component"]

            if type(components) is dict:
                monomers.append(components["Protein"]["@frameid"])
                inAddition.append(components["Protein"]["@frameid"])
                if "coefficient" in components:
                    coef = components["coefficient"]["#text"]
                    count.append(int(coef))
                else:
                    count.append(1)
            else:
                for protein in components:
                    monomers.append(protein["Protein"]["@frameid"])
                    inAddition.append(protein["Protein"]["@frameid"])
                    if "coefficient" in protein:
                        coef = protein["coefficient"]["#text"]
                        count.append(int(coef))
                    else:
                        count.append(1)
        else:
            inAddition.append(components["@frameid"])

    genes = getGenes(inAddition)

    data = [monomers, count, genes]
    
    return data

In [ ]:
def getProteinComplexes(tree):
    result = []
    
    complexes = tree["ptools-xml"]["Reaction"]["enzymatic-reaction"]["Enzymatic-Reaction"]
    
    if type(complexes) is dict:
        result.append(complexes["enzyme"]["Protein"]["@frameid"])
    else:
        for complexR in complexes:
             result.append(complexR["enzyme"]["Protein"]["@frameid"])
                
    return result